# Week_3 Assignment_2: Анализ текстов

https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/NdyLO/analiz-tiekstov

Данное задание основано на материалах лекций по методу опорных векторов.

#### Вы научитесь:
- находить оптимальные параметры для метода опорных векторов
- работать с текстовыми данными

#### Введение
Метод опорных векторов (Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF). Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

#### Данные
Как мы уже говорили выше, линейные методы часто применяются для решения различных задач анализа текстов. В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.

#### Реализация в Scikit-Learn
Для начала вам потребуется загрузить данные. В этом задании мы воспользуемся одним из датасетов, доступных в scikit-learn'е — 20 newsgroups. Для этого нужно воспользоваться модулем datasets:

    from sklearn import datasets

    newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )
После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе [sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

Реализация SVM-классификатора находится в классе [sklearn.svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html?highlight=sklearn%20svm%20svc#sklearn.svm.SVC). Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:

    feature_mapping = vectorizer.get_feature_names()
    print feature_mapping[i]

Подбор параметров удобно делать с помощью класса [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=sklearn%20model_selection%20gridsearchcv#sklearn.model_selection.GridSearchCV). Пример использования:

    grid = {'C': np.power(10.0, np.arange(-5, 6))}
    cv = KFold(n_splits=5, shuffle=True, random_state=241)
    clf = svm.SVC(kernel='linear', random_state=241)
    gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
    gs.fit(X, y)

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

    for a in gs.grid_scores_:
        # a.mean_validation_score — оценка качества по кросс-валидации
        # a.parameters — значения параметров

#### Инструкция по выполнению
1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут
2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по **всем** данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.
3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.
5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

## Ответы на вопросы задания

In [19]:
import pandas as pd
import numpy as np
import sklearn

### 1. Загрузите объекты из новостного датасета
относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [85]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

data = newsgroups.data
y = newsgroups.target
y

array([0, 0, 1, ..., 1, 1, 0])

In [3]:
type(y)

numpy.ndarray

In [4]:
y.size

1786

### 2. Вычислите TF-IDF-признаки для всех текстов
Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer as tv
vectorizer = tv()
X = vectorizer.fit_transform(data)
print(X.shape)

(1786, 28382)


### 3. Подберите минимальный лучший параметр C
из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

хорошая объяснялка в помощь: [тыц](https://vc.ru/ml/147132-kak-avtomaticheski-podobrat-parametry-dlya-modeli-mashinnogo-obucheniya-ispolzuem-gridsearchcv)

In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV as GSCV

In [30]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GSCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [36]:
gs.score(X,y)    # смотрим какая у нас получилась точность на оптимальном параметре

0.9994400895856663

In [46]:
gs.best_params_  # смотрим значение оптимального параметра

{'C': 1.0}

In [79]:
res = pd.DataFrame(gs.cv_results_)
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.320137,0.177012,0.901082,0.040442,1e-05,{'C': 1e-05},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
1,3.258248,0.079198,0.876305,0.007541,0.0001,{'C': 0.0001},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
2,3.411352,0.171161,0.887359,0.020671,0.001,{'C': 0.001},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
3,3.271040,0.057098,0.897252,0.017531,0.01,{'C': 0.01},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
4,3.008503,0.309560,0.819140,0.098440,0.1,{'C': 0.1},0.958101,0.949580,0.957983,0.935574,0.949580,0.950164,0.008218,7
5,1.654424,0.022544,0.439559,0.016354,1,{'C': 1.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
6,1.634081,0.032000,0.436151,0.008409,10,{'C': 10.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
7,1.634955,0.037586,0.430956,0.011965,100,{'C': 100.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
8,1.695803,0.106676,0.454302,0.024866,1000,{'C': 1000.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
9,1.651695,0.050469,0.440348,0.009373,10000,{'C': 10000.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1


### 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге

In [90]:
clf = SVC(C=1.0, kernel='linear', random_state=241)
clf.fit(X, y)

SVC(kernel='linear', random_state=241)

### 5. Найдите 10 слов с наибольшим абсолютным значением веса
(веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке

In [91]:
W = np.sort(np.absolute(clf.coef_.toarray()))  # отсортированные по модулю веса слов
W

array([[0.        , 0.        , 0.        , ..., 1.25468995, 1.9203794 ,
        2.66316479]])

In [92]:
indexes = np.argsort(np.absolute(clf.coef_.toarray()))  # отсортированные по модулю весов индексы слов
indexes

array([[ 7801, 21437,  9144, ...,  5088, 12871, 24019]])

In [93]:
top_10_indexes = np.array(indexes[0,-10:], dtype=int)  # последние 10 индексов
top_10_indexes

array([22936, 15606,  5776, 21850, 23673, 17802,  5093,  5088, 12871,
       24019])

In [96]:
list = [vectorizer.get_feature_names()[i] for i in top_10_indexes]  # переводим признаки в слова
print(np.sort(list))  # выводим топ-10 слов (по возрастанию индекса, как в чартовой дюжине MTV)

['atheism' 'atheists' 'bible' 'god' 'keith' 'moon' 'religion' 'sci' 'sky'
 'space']
